# Benchmarking classification models on MNIST-1D

In [47]:
# Run this cell if you are in Colab 
# (otherwise make sure to do python -m pip install mnist1d first)

#!python -m pip install git+https://github.com/michael-beebe/mnist1d.git@master
    
# Download repo directly (gives access to notebooks/models.py and notebooks/train.py)
# !git clone https://github.com/michael-beebe/mnist1d

In [48]:
import sys
sys.path.append('./mnist1d/notebooks/')
sys.path.append('./mnist1d/mnist1d/')

from models import ConvBase, MLPBase, LinearBase, GRUBase
from train import get_model_args, train_model
from models import ResNet1DBase, DilatedCNNBase
from tcn import train_TCN, TCNBase
from mnist1d.utils import set_seed, plot_signals, ObjectView, from_pickle
import torch

# DEVICE = 'mps' if torch.backends.mps.is_available() else 'cpu'
# DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
DEVICE = 'cpu'

In [49]:
# Load MNIST-1D
# (loading from the web to make running in a Colab easier)

from urllib.request import urlopen
import pickle

url = 'https://github.com/michael-beebe/mnist1d/raw/master/mnist1d_data.pkl'
data = pickle.load(urlopen(url))

## Run the benchmarks

In [ ]:
# Note: if you run on a GPU, the walltimes will be _much_ lower

args = get_model_args()
args.total_steps = 4000
args.device = DEVICE

set_seed(args.seed)
model = ConvBase(output_size=args.output_size)
results_cnn = train_model(data, model, args)
print()

set_seed(args.seed)
model = GRUBase(input_size=1, output_size=args.output_size)
results_gru = train_model(data, model, args)
print()

set_seed(args.seed)
model = MLPBase(args.input_size, args.output_size)
results_mlp = train_model(data, model, args)
print()

set_seed(args.seed)
model = LinearBase(args.input_size, args.output_size)
results_lin = train_model(data, model, args)

#############################################
#                New models
#############################################
# --- ResNet1D
set_seed(args.seed)
model = ResNet1DBase(input_size=args.input_size, output_size=args.output_size)
results_resnet = train_model(data, model, args)
print()

# --- TCN
set_seed(args.seed)
model = TCNBase(input_size=args.input_size, output_size=args.output_size, num_channels=[64, 128, 10], dropout=0.01)
results_tcn = train_TCN(data, model, args)
print()

# --- DilatedCNN
set_seed(args.seed)
model = DilatedCNNBase(input_size=args.input_size, output_size=args.output_size)
results_dilated_cnn = train_model(data, model, args)
print()


In [51]:
# Shuffle the pixels

import numpy as np

np.random.seed(42)
shuffled_order = np.random.permutation(data['x'].shape[1])

data_shuff = data.copy()
data_shuff['x'] = data_shuff['x'][:, shuffled_order]
data_shuff['x_test'] = data_shuff['x_test'][:, shuffled_order]

In [ ]:
set_seed(args.seed)
model = ConvBase(output_size=args.output_size)
results_cnn_shuff = train_model(data_shuff, model, args)
print()

set_seed(args.seed)
model = GRUBase(input_size=1, output_size=args.output_size)
results_gru_shuff = train_model(data_shuff, model, args)
print()

set_seed(args.seed)
model = MLPBase(args.input_size, args.output_size)
results_mlp_shuff = train_model(data_shuff, model, args)
print()

set_seed(args.seed)
model = LinearBase(args.input_size, args.output_size)
results_lin_shuff = train_model(data_shuff, model, args)

#############################################
#                New models
#############################################
# --- ResNet1D
set_seed(args.seed)
model = ResNet1DBase(input_size=args.input_size, output_size=args.output_size)
results_resnet_shuff = train_model(data_shuff, model, args)
print()

# --- TCN
set_seed(args.seed)
model = TCNBase(input_size=args.input_size, output_size=args.output_size, num_channels=[64, 128, 10], dropout=0.01)
results_tcn_shuff = train_TCN(data_shuff, model, args)
print()

# --- DilatedCNN
set_seed(args.seed)
model = DilatedCNNBase(input_size=args.input_size, output_size=args.output_size)
results_dilated_cnn_shuff = train_model(data_shuff, model, args)
print()

## Construct the figure

In [ ]:
import pylab as plt

# Figure style (loading from the web to make running in a Colab easier)
plt.style.use('https://github.com/michael-beebe/mnist1d/raw/master/notebooks/mpl_style.txt')

fig, axs = plt.subplots(ncols=2, figsize=(3.25, 2.25))

steps = 6000
eval_every = 250
t = range(0, steps + 1, eval_every)

axs[0].plot(t, results_lin['train_acc'], '-', color='tab:red', label='Logistic')
axs[0].plot(t, results_lin_shuff['train_acc'], '--', color='tab:red')

axs[0].plot(t, results_mlp['train_acc'], '-', color='tab:green', label='MLP', clip_on=False)
axs[0].plot(t, results_mlp_shuff['train_acc'], '--', color='tab:green', clip_on=False)

axs[0].plot(t, results_cnn['train_acc'], '-', color='tab:blue', label='CNN', clip_on=False)
axs[0].plot(t, results_cnn_shuff['train_acc'], '--', color='tab:blue', clip_on=False)

axs[0].plot(t, results_gru['train_acc'], '-', color='tab:orange', label='GRU', clip_on=False)
axs[0].plot(t, results_gru_shuff['train_acc'], '--', color='tab:orange', clip_on=False)

#############################################
axs[0].plot(t, results_resnet['train_acc'], '-', color='tab:purple', label='ResNet1D', clip_on=False)
axs[0].plot(t, results_resnet_shuff['train_acc'], '--', color='tab:purple', clip_on=False)

axs[0].plot(t, results_tcn['train_acc'], '-', color='tab:brown', label='TCN', clip_on=False)
axs[0].plot(t, results_tcn_shuff['train_acc'], '--', color='tab:brown', clip_on=False)

axs[0].plot(t, results_dilated_cnn['train_acc'], '-', color='tab:pink', label='DilatedCNN', clip_on=False)
axs[0].plot(t, results_dilated_cnn_shuff['train_acc'], '--', color='tab:pink', clip_on=False)

axs[0].plot(t, [-1] * len(t), 'k--', label='After shuffling')
#############################################

###

axs[1].plot(t, results_lin['test_acc'], '-', color='tab:red', label='logistic')
axs[1].plot(t, results_lin_shuff['test_acc'], '--', color='tab:red')

axs[1].plot(t, results_mlp['test_acc'], '-', color='tab:green', label='mlp')
axs[1].plot(t, results_mlp_shuff['test_acc'], '--', color='tab:green')

axs[1].plot(t, results_cnn['test_acc'], '-', color='tab:blue', label='cnn')
axs[1].plot(t, results_cnn_shuff['test_acc'], '--', color='tab:blue')

axs[1].plot(t, results_gru['test_acc'], '-', color='tab:orange', label='gru')
axs[1].plot(t, results_gru_shuff['test_acc'], '--', color='tab:orange')


#############################################
axs[1].plot(t, results_resnet['test_acc'], '-', color='tab:purple', label='ResNet1D')
axs[1].plot(t, results_resnet_shuff['test_acc'], '--', color='tab:purple')

axs[1].plot(t, results_tcn['test_acc'], '-', color='tab:brown', label='TCN')
axs[1].plot(t, results_tcn_shuff['test_acc'], '--', color='tab:brown')

axs[1].plot(t, results_dilated_cnn['test_acc'], '-', color='tab:pink', label='DilatedCNN')
axs[1].plot(t, results_dilated_cnn_shuff['test_acc'], '--', color='tab:pink')
#############################################


axs[1].plot(t, [95.8] * len(t), 'k-')
axs[1].text(0, 91, 'Human')

axs[0].set_title('Training accuracy')
axs[1].set_title('Test accuracy')
axs[1].set_xlabel("Training step")
axs[0].set_xlabel("Training step")
axs[0].set_ylabel("Accuracy (%)")
axs[0].legend()
axs[0].set_ylim(0, 100)
axs[1].set_ylim(0, 100)

fig.text(0, .95, 'a', fontsize=8, weight='bold')
fig.text(.5, .95, 'b', fontsize=8, weight='bold')

plt.show()

fig.savefig('figures/benchmark.png', dpi=300)
fig.savefig('figures/benchmark.pdf')